# Behavioral Cloning

In [1]:
# Run only once, to solve the conflict with ROS
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

## Simple Model

In [2]:
import os
import csv
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import cv2
from sklearn.utils import shuffle

samples = []
# with open('data/driving_log.csv') as csvfile:
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
        
images = []
measurements = []
for line in samples:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = 'data/IMG/' + filename
    image = plt.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
X_train = np.array(images)
y_train = np.array(measurements)

In [20]:
from keras.models import Sequential
from keras.layers import Cropping2D, Lambda, Flatten, Dense

# Set our batch size
batch_size=32

model = Sequential()
# model.add(Cropping2D(cropping=((55,25), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
# TODO: 
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=3)

model.save('model.h1')
# exit()


/home/ss/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 1357 samples, validate on 340 samples
Epoch 1/3
1357/1357 [==============================] - 1s 669us/step - loss: 11.2480 - val_loss: 3.5623
Epoch 2/3
1357/1357 [==============================] - 0s 288us/step - loss: 2.7329 - val_loss: 1.0701
Epoch 3/3
1357/1357 [==============================] - 0s 308us/step - loss: 1.2451 - val_loss: 0.6374


## Data Augmentation

In [3]:
import os
import csv
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.utils import shuffle

samples = []
# with open('data/driving_log.csv') as csvfile:
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
        
        
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, 
                                                     test_size=0.2)


In [17]:
def multi_cam(batch_sample):
    correction = 0.1
    
    multi_cam_images, multi_cam_angles = [], []
    for i in range(0, 3):
        name = 'data/IMG/' + batch_sample[i].split('/')[-1]
        image = plt.imread(name)
        # if using the left/right/center camera
        if(i == 1): 
            angle = float(batch_sample[3]) + correction
        elif(i == 2): 
            angle = float(batch_sample[3]) - correction
        else: 
            angle = float(batch_sample[3])
        multi_cam_images.append(image)
        multi_cam_angles.append(angle)
    return multi_cam_images, multi_cam_angles


def flip_img(images, angles):
    fliped_images, fliped_angles = [], []
    for image in images:
        fliped_images.append(image)
        fliped_images.append(np.fliplr(image))
    for angle in angles:
        fliped_angles.append(angle)
        fliped_angles.append(angle * (-1))
    return fliped_images, fliped_angles


def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images, angles = [], []
            for batch_sample in batch_samples:
                
                # Using all three cameras, output 3*1 array of images and angles
                multi_cam_images, multi_cam_angles = multi_cam(batch_sample)
                # flip all the camera images, output 6*1 array of images and angles
                fliped_images, fliped_angles = flip_img(multi_cam_images, multi_cam_angles)
                
                output_images = fliped_images
                output_angles = fliped_angles
                images.extend(output_images)
                angles.extend(output_angles)
                
                #name = 'data/IMG/'+batch_sample[0].split('/')[-1]
                #center_image = plt.imread(name)
                #center_angle = float(batch_sample[3])
                #images.append(center_image)
                #angles.append(center_angle)
                
            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)



## Simple Network

In [32]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Cropping2D, Lambda, Conv2D, MaxPooling2D, Flatten, Dense

# Set our batch size
batch_size = 64

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

model = Sequential()
model.add(Cropping2D(cropping=((55,25), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x: x/255.0 - 0.5))
# TODO: 
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=5)

model.save('model.h1')
# exit()

/home/ss/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1357 samples, validate on 340 samples
Epoch 1/5
1357/1357 [==============================] - 2s 1ms/step - loss: 98.4720 - val_loss: 59.2222
Epoch 2/5
1357/1357 [==============================] - 1s 462us/step - loss: 41.0886 - val_loss: 16.6158
Epoch 3/5
1357/1357 [==============================] - 1s 463us/step - loss: 13.2054 - val_loss: 3.6572
Epoch 4/5
1357/1357 [==============================] - 1s 464us/step - loss: 5.2432 - val_loss: 8.8218
Epoch 5/5
1357/1357 [==============================] - 1s 467us/step - loss: 3.8930 - val_loss: 3.7457


## LeNet Architecture

In [33]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Cropping2D, Lambda, Conv2D, MaxPooling2D, Flatten, Dense

# Set our batch size
batch_size = 32

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0))))
model.add(Conv2D(6, (5, 5), activation="relu", strides=(1, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, (5, 5), activation="relu", strides=(1, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, 
                    steps_per_epoch=np.ceil(len(train_samples)/batch_size), 
                    validation_data=validation_generator, 
                    validation_steps=np.ceil(len(validation_samples)/batch_size), 
                    epochs=10, verbose=1)

model.save('model.h3')
print("model saved")
# exit()

Epoch 1/10
43/43 [==============================] - 6s 130ms/step - loss: 0.4058 - val_loss: 0.0190
Epoch 2/10
43/43 [==============================] - 4s 103ms/step - loss: 0.0175 - val_loss: 0.0147
Epoch 3/10
43/43 [==============================] - 4s 102ms/step - loss: 0.0152 - val_loss: 0.0135
Epoch 4/10
43/43 [==============================] - 4s 102ms/step - loss: 0.0139 - val_loss: 0.0127
Epoch 5/10
43/43 [==============================] - 4s 103ms/step - loss: 0.0130 - val_loss: 0.0122
Epoch 6/10
43/43 [==============================] - 4s 103ms/step - loss: 0.0120 - val_loss: 0.0120
Epoch 7/10
43/43 [==============================] - 4s 102ms/step - loss: 0.0110 - val_loss: 0.0116
Epoch 8/10
43/43 [==============================] - 4s 103ms/step - loss: 0.0101 - val_loss: 0.0114
Epoch 9/10
43/43 [==============================] - 4s 103ms/step - loss: 0.0093 - val_loss: 0.0112
Epoch 10/10
43/43 [==============================] - 4s 103ms/step - loss: 0.0085 - val_loss: 0.0111

## Nvidia End-to-End CNN Architecture

In [12]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Cropping2D, Lambda, Conv2D, Flatten, Dense

# Set our batch size
batch_size = 32

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0))))
model.add(Conv2D(24, (5, 5), activation="relu", strides=(2, 2)))
model.add(Conv2D(36, (5, 5), activation="relu", strides=(2, 2)))
model.add(Conv2D(48, (5, 5), activation="relu", strides=(2, 2)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

with tf.device('/GPU:0'):
    model.compile(loss='mse', optimizer='adam')
    model.fit_generator(train_generator, 
                        steps_per_epoch=np.ceil(len(train_samples)/batch_size), 
                        validation_data=validation_generator, 
                        validation_steps=np.ceil(len(validation_samples)/batch_size), 
                        epochs=5, verbose=1)

model.save('model.h5')
# exit()

Epoch 1/5
43/43 [==============================] - 1s 34ms/step - loss: 0.0122 - val_loss: 0.0093
Epoch 2/5
43/43 [==============================] - 1s 21ms/step - loss: 0.0101 - val_loss: 0.0093
Epoch 3/5
43/43 [==============================] - 1s 23ms/step - loss: 0.0102 - val_loss: 0.0086
Epoch 4/5
43/43 [==============================] - 1s 23ms/step - loss: 0.0092 - val_loss: 0.0076
Epoch 5/5
43/43 [==============================] - 1s 23ms/step - loss: 0.0079 - val_loss: 0.0092
